## Imports

In [ ]:
!pip install farm-haystack transformers ollama
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!pip install sentence-transformers
!pip install "farm-haystack[inference]"

In [ ]:
from haystack.document_stores import InMemoryDocumentStore
from haystack.nodes import EmbeddingRetriever
from haystack.schema import Document
import ollama

## Prepare key terms embeddings

In [ ]:
document_store = InMemoryDocumentStore(embedding_dim=384)

key_terms = open('terms.csv').read().split('\n')

documents = [Document(content=term) for term in key_terms]

document_store.write_documents(documents)

In [ ]:
retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model="sentence-transformers/all-MiniLM-L6-v2",
    use_gpu=False 
)

document_store.update_embeddings(retriever)

## Prepare question embedding and question for Llama3.2

In [56]:
def get_relevant_key_terms(question, top_k=20):
    results = retriever.retrieve(query=question, top_k=top_k)
    return [doc.content for doc in results]

client = ollama.Client()

def ask_llama(question, relevant_key_terms):
    prompt = f"""
        Here is a search prompt by a user of the UN digital library:
        Question: {question}
        Choose the three most relevant key terms to the question (separated by semi-colons):
        Key Terms: {relevant_key_terms}
        Output only the selected key terms, separated by semi-colons, nothing else.
        """

    return client.chat(model="llama3.2", messages=[{"role": "user", "content": prompt}])['message']['content']


## Ask question

In [57]:
question = "TRADE PROMOTION, FACILITATION AND CUSTOMS ADMINISTRATION"

In [58]:
relevant_key_terms = get_relevant_key_terms(question)
relevant_key_terms = ";".join(relevant_key_terms)
print("Relevant Key Terms:", relevant_key_terms)

Batches: 100%|██████████| 1/1 [00:00<00:00, 101.79it/s]

Relevant Key Terms: "TRADE PROMOTION, FACILITATION AND CUSTOMS ADMINISTRATION";TRADE FACILITATION;CUSTOMS ADMINISTRATION;TRADE PROMOTION;TRADE FACILITATION CENTRES;INTERNATIONAL TRADE;IMPORT PROMOTION;FOREIGN TRADE;IMPORT PROCUREMENT;TRADE ASSOCIATIONS;TRADE REGULATION;EXPORT PROMOTION;TRADE POLICY;CUSTOMS ESCORTS;CUSTOMS UNIONS;GENERAL INTERNATIONAL TRADE AND TRADE POLICY;INDUSTRIAL PROMOTION;TRADE REPRESENTATION ABROAD;FOREIGN TRADE POLICY;RETAIL TRADE


In [59]:
is_answer_valid = False
while not is_answer_valid:
    answer = ask_llama(question, relevant_key_terms)
    print("Answer:", answer)

    answer_split = [term.strip() for term in answer.split(';')]
    is_answer_valid = isinstance(answer, str) and len(answer_split) == 3
    is_answer_valid = is_answer_valid and all([term.strip() in relevant_key_terms for term in answer.split(',')])
    print(is_answer_valid)

Answer: TRADE PROMOTION;TRADE FACILITATION;CUSTOMS ADMINISTRATION
False
Answer: TRADE FACILITATION;CUSTOMS ADMINISTRATION;TRADE PROMOTION
True


In [60]:
print(answer_split)
output = f"subjectheading:[{answer_split[0]}]subjectheading:[{answer_split[1]}]subjectheading:[{answer_split[2]}]"
print(output)

['TRADE FACILITATION', 'CUSTOMS ADMINISTRATION', 'TRADE PROMOTION']
subjectheading:[TRADE FACILITATION]subjectheading:[CUSTOMS ADMINISTRATION]subjectheading:[TRADE PROMOTION]
